In [ ]:
import pandas as pd
import xarray as xr
import holoviews as hv
import numpy as np
from scipy.stats import linregress
from matplotlib import pyplot as plt

hv.extension('bokeh')

In [ ]:
fix = pd.read_excel('/Volumes/LtgSSD/arm-ccn-fix/modeloutput.xlsx', skiprows=1)

In [ ]:
fix.columns

In [ ]:
corrected_CCN = fix.set_index('Timestamp (UTC)')['CCN @ SS = 0.1% (cm-3)']

In [ ]:
wrong = xr.open_mfdataset('/Volumes/LtgSSD/arm-ccn-avg/*.nc')

In [ ]:
wrong_filt = wrong.isel(time=((wrong.supersaturation_calculated <= 0.15)).compute())

In [ ]:
wrong_curve = hv.Scatter((wrong_filt.time, wrong_filt.N_CCN), kdims=['Time'], vdims=['N_CCN (cm^-3)'], label='AOS CCN Instrument').opts(width=800, height=400, color='red', line_width=2)

In [ ]:
fixed_curve = hv.Scatter((corrected_CCN.index, corrected_CCN.values), kdims=['Time'], vdims=['N_CCN (cm^-3)'], label='pyrcel model CCN').opts(width=800, height=400, color='blue', line_width=2, title='CCN Timeseries, supersaturation = 0.1%')

In [ ]:
wrong_curve * fixed_curve

In [ ]:
hourly_resample = wrong_filt.N_CCN.resample(time='1h').mean().compute()

In [ ]:
wrong_mask_to_model = hourly_resample.isel(time=hourly_resample.time.isin(corrected_CCN.index).compute())

In [ ]:
wrong_mask_to_model

In [ ]:
model_matching_obs_times = corrected_CCN.loc[wrong_mask_to_model.time.values]

In [ ]:
all_nonnan = (~np.isnan(model_matching_obs_times.values)) & (~np.isnan(wrong_mask_to_model.data))

In [ ]:
res = linregress(model_matching_obs_times.values[all_nonnan], wrong_mask_to_model.data[all_nonnan])

In [ ]:
m = res.slope
b = res.intercept
r = res.rvalue
p = res.pvalue
stderr = res.stderr
int_stderr = res.intercept_stderr 

In [ ]:
hv.Scatter((model_matching_obs_times, wrong_mask_to_model)).opts(width=800, height=400, color='green', xlabel='pyrcel model', ylabel='AOS CCN', title=f'Averaged Hourly AOS CCN observation vs pyrcel model value at 1.05-1.35%SS\nm = {m:.2f}, b = {b:.2f}, r^2 = {r**2:.2f}, p = {p:.2f}')

In [ ]:
wrong.supersaturation_calculated.plot.hist(bins=100)
plt.xlabel('Supersaturation (%)')
plt.ylabel('Frequency')
plt.title('Supersaturation Distribution in AOS CCN Data')